In [1]:
# from goorawling import Goorawling
import os

# Change current working directory to parent directory
# Save the original working directory
original_cwd = os.getcwd()

# Change to parent directory
os.chdir("..")

from tools import *

In [2]:
import pandas as pd

# Read the CSV file for Ho Chi Minh restaurants
df = pd.read_csv('data/tripadvisor_restaurant/Ha Noi.csv')
df.head(1).T

def add_address_string_column(df):
    import ast
    # Convert each address_obj string to dict and extract 'address_string'
    df['address_string'] = df['address_obj'].apply(lambda x: ast.literal_eval(x).get('address_string'))
    return df

# Add the new column
df = add_address_string_column(df)

# Save the updated DataFrame to the same CSV file
df.to_csv('data/tripadvisor_restaurant/Ha Noi.csv', index=False)

In [3]:
df[['name','address_string']]

,name,address_string
0,Hoang's Restaurant,"54 Hang Buom Street, Hoan Kiem, Hanoi 100000 V..."
1,Hong Hoai's Restaurant,"20 Bat Dan, Hoan Kiem, Hanoi 100000 Vietnam"
2,Essence Restaurant Original (since 2011),"38A Tran Phu Street, Ba Dinh, Hanoi 100000 Vie..."
3,Banh Mi 25,"25 Hang Ca, Hoan Kiem, Hanoi 10000 Vietnam"
4,Quan An Ngon,"18 P. Phan B_i Chau, Hoan Kiem, Hanoi 111103 V..."
...,...,...
995,Chả Ca Ca Cuống,"17 Cha Ca, Hanoi 123456 Vietnam"
996,Amy Indian Restaurant,"10 P. Hang Thi_c, Hoan Kiem, Hanoi 00120 Vietnam"
997,Zpizza,"27 Ly Thai To, Hoan Kiem, Hanoi 10000 Vietnam"
998,Pizza 4P's Lotte West Lake Hanoi,"272 Vo Chi Cong street., Hanoi 11210 Vietnam"


In [4]:
# df

In [8]:
def get_name_address_list(df):
    return [" ".join([str(row['name']), str(row['address_string'])]) for _, row in df.iterrows()]

name_address_list = get_name_address_list(df)
# name_address_list

In [3]:
def get_restaurant_link(driver,restaurant_string:str):
    """
    Searches for a restaurant on Google Maps and returns the link to the first result.
    """
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.fontBodyMedium.searchboxinput.xiQnY')))
    input_goo = driver.find_element(By.CSS_SELECTOR, '.fontBodyMedium.searchboxinput.xiQnY')
    find_button = driver.find_element(By.CSS_SELECTOR, '.mL3xi')
    time.sleep(1)
    input_goo.clear()
    input_goo.send_keys(restaurant_string)
    find_button.click()
    time.sleep(3)
    try: 
        get_first_restaurant = driver.find_element(By.CSS_SELECTOR, '.hfpxzc')
        restaurant_link =  get_first_restaurant.get_attribute('href')
    except NoSuchElementException:
        restaurant_link = driver.current_url
    time.sleep(2)
    return change_language_to_vietnamese(restaurant_link)

In [5]:
def extract_lat_lon_from_url(url):
    """
    Extract latitude and longitude from a Google Maps URL.
    Returns (lat, lon) as strings, or (None, None) if not found.
    """
    try:
        # Example: .../@21.036118,105.8460021,17z/...
        at_part = url.split('/@')[1].split('/')[0]
        lat, lon = at_part.split(',')[:2]
        return lat, lon
    except Exception:
        return None, None

# Example usage:
exam = 'https://www.google.com/maps/place/Banh+Mi+25/@21.036118,105.8460021,17z/data=!3m1!4b1!4m6!3m5!1s0x3135ab74bb3716b5:0xebfbc0d84354deb3!8m2!3d21.036113!4d105.848577!16s%2Fg%2F11tw_4rt41?entry=ttu&g_ep=EgoyMDI1MDYzMC4wIKXMDSoASAFQAw%3D%3D&hl=vi'
lat, lon = extract_lat_lon_from_url(exam)
print(lat, lon)  # Output: 21.036118 105.8460021

21.036118 105.8460021


In [6]:

df.shape

(1000, 32)

In [ ]:
import threading

def scrape_google_maps_links(df, num_threads=4, output_name="output.csv"):
    from concurrent.futures import ThreadPoolExecutor, as_completed
    from selenium import webdriver

    search_list = df[['name', 'address_string']].astype(str).agg(' '.join, axis=1).tolist()
    write_lock = threading.Lock()
    progress_lock = threading.Lock()
    total = len(search_list)
    progress_counter = [0]  # Use a list for mutability in threads

    # Write header once at the start (overwrite file)
    with open(output_name, "w", encoding="utf-8") as f:
        f.write("restaurant_id,restaurant_link\n")

    def scrape_link(idx, restaurant_string):
        driver = webdriver.Chrome(options=chrome_options)
        driver.get("https://www.google.com/maps")
        time.sleep(2)
        try:
            link = get_restaurant_link(driver, restaurant_string)
        except Exception:
            link = None
        driver.quit()
        row = df.iloc[idx]
        restaurant_id = row['restaurant_id']
        with write_lock:
            with open(output_name, "a", encoding="utf-8") as f:
                f.write(f"{restaurant_id},{link}\n")
        with progress_lock:
            progress_counter[0] += 1
            print(f"Progress: {progress_counter[0]} / {total}")
        return (restaurant_id, link)

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(scrape_link, idx, s) for idx, s in enumerate(search_list)]
        for future in as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Error: {e}")

# Example usage:
# scrape_google_maps_links(df, num_threads=4, output_name="data/tripadvisor_restaurant/Ha_Noi_with_links.csv")

In [8]:
scrape_google_maps_links(df[:5], num_threads=5, output_name="data/tripadvisor_restaurant/Ha_Noi_with_links.csv")

Progress: 1 / 5
Progress: 2 / 5
Progress: 3 / 5
Progress: 4 / 5
Progress: 5 / 5


In [9]:
# link = 

In [31]:
driver = webdriver.Chrome(options=chrome_options)
actions = ActionChains(driver)
driver.get('https://www.google.com/maps')

In [23]:
print(name_address_list[3000])

Shamoji Robata Yaki 135 Nguy_n D_c C_nh, Tan Phong, Qu_n 7 135 Nguyễn Đức Cảnh, Tân Phong, Quận 7, Thành phố Hồ Chí Minh, Vietnam, Ho Chi Minh City 756600 Vietnam


In [32]:
input_goo = driver.find_element(By.CSS_SELECTOR, '.fontBodyMedium.searchboxinput.xiQnY')
find_button = driver.find_element(By.CSS_SELECTOR, '.mL3xi')

In [29]:
input_goo.clear()


In [ ]:
for i in range(5):
    input_goo.clear()
    input_goo.send_keys(name_address_list[3000:3000 + i])
    find_button.click()
    time.sleep(5)
    try: 
        get_first_restaurant = driver.find_element(By.CSS_SELECTOR, '.hfpxzc')
        restaurant_link =  get_first_restaurant.get_attribute('href')
    except NoSuchElementException:
        restaurant_link = driver.current_url
    time.sleep(2)
    print(change_language_to_vietnamese(restaurant_link))

# input_goo.clear()

https://www.google.com/maps/place/Shamoji+Robata+Yaki+-+Japanese+Izakaya/@10.7239543,106.7064369,17z/data=!3m1!4b1!4m6!3m5!1s0x31752fc8104cfea1:0x38e1bd182fb4869d!8m2!3d10.723949!4d106.7090118!16s%2Fg%2F11sqkr6n2x?entry=ttu&g_ep=EgoyMDI1MDYwMy4wIKXMDSoASAFQAw%3D%3D&hl=vi
https://www.google.com/maps/search/Shamoji+Robata+Yaki+135+Nguy_n+D_c+C_nh,+Tan+Phong,+Qu_n+7+135+Nguy%E1%BB%85n+%C4%90%E1%BB%A9c+C%E1%BA%A3nh,+T%C3%A2n+Phong,+Qu%E1%BA%ADn+7,+Th%C3%A0nh+ph%E1%BB%91+H%E1%BB%93+Ch%C3%AD+Minh,+Vietnam,+Ho+Chi+Minh+City+756600+Vietnam/@10.7239543,106.7064369,17z?entry=ttu&g_ep=EgoyMDI1MDYwMy4wIKXMDSoASAFQAw%3D%3D&hl=vi
https://www.google.com/maps/place/Nori+-+Modern+Izakaya/data=!4m7!3m6!1s0x31752f3f52ce6b9d:0x2f7c030139c58cdb!8m2!3d10.773996!4d106.696912!16s%2Fg%2F11vkz338lg!19sChIJnWvOUj8vdTER24zFOQEDfC8?authuser=0&hl=vi&rclk=1
https://www.google.com/maps/place/Nori+-+Modern+Izakaya/data=!4m7!3m6!1s0x31752f3f52ce6b9d:0x2f7c030139c58cdb!8m2!3d10.773996!4d106.696912!16s%2Fg%2F11vkz338lg!1

In [21]:
find_button = driver.find_element(By.CSS_SELECTOR, '.mL3xi')
find_button.click()

In [24]:
try: 
    get_first_restaurant = driver.find_element(By.CSS_SELECTOR, '.hfpxzc')
    restaurant_link =  get_first_restaurant.get_attribute('href')
except NoSuchElementException:
    restaurant_link = driver.current_url

In [25]:
change_language_to_vietnamese(restaurant_link)

'https://www.google.com/maps/place/Shamoji+Robata+Yaki+-+Japanese+Izakaya/@10.7239543,106.7064369,17z/data=!3m1!4b1!4m6!3m5!1s0x31752fc8104cfea1:0x38e1bd182fb4869d!8m2!3d10.723949!4d106.7090118!16s%2Fg%2F11sqkr6n2x?entry=ttu&g_ep=EgoyMDI1MDYwMy4wIKXMDSoASAFQAw%3D%3D&hl=vi'